## Разделим текст на сообщения

In [8]:
import re

In [9]:
text = open('ling_emails.txt','r').read()
messages = re.findall('ЭТОМОЁПИСЬМОНОМЕР:[0-9]*(.+?)ЭТОКОНЕЦПИСЬМАНОМЕР', text, flags = re.DOTALL)

## Уберем всякую ерунду из писем (цитаты, переадресовки и т.п.)

In [325]:
def cleaner(text):
    lines0 = text.split('\n')
    n = 0
    lines = []
    for i in lines0:
        if i != '':
            lines.append(i)
    i = 0
    j = 0
    while i != len(lines):
        if len(re.findall('[0-9]*:[0-9]* пользователь', lines[i])) > 0 or len(re.findall('On Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday, [0-9]+ September|October|November|December|January|February|March|April|May|June|July|August [0-9]+, ', lines[i])) > 0 or len(re.findall('_____', lines[i])) > 0 or len(re.findall('Отправлено со', lines[i])) > 0 or len(re.findall('Sent from', lines[i]))>0 or len(re.findall('> ', lines[i])) > 0 or len(re.findall('---------- Пересылаемое сообщение ----------', lines[i]))>0:
            lines = lines[:i]
            lines_str = ''
            for j in lines:
                lines_str += j
                lines_str += ' '
            break
        if len(re.findall('Копия:', lines[i])) > 0:
            lines = lines[i+1:]
            lines_str = ''
            for j in lines:
                lines_str += j
                lines_str += ' '
            break
        i += 1
    lines_str = ''
    for j in lines:
        lines_str += j
        lines_str += ' '
    return lines_str

In [64]:
r = re.search('aa', 'nmlaa')
r.end()

5

In [34]:
def rev(x):
    y = ''
    for i in reversed(x):
        y += i
    return y

In [203]:
def lstrip(x):
    punct = []
    while len(x) > 0 and x[0] in ['?', '!', '.', ')', '(', ';', ':', '"', '-', '—', '*' , ',']:
        if x[0] == '.':
            punct.append(re.match('\.+', x).group(0)) #нашли все многоточия (в начале и в конце)
            x = x.lstrip('\.+') #убрали многоточие
        elif x[0] == '?' or x[0] == '!':
            punct.append(re.match('((!|\?)+)', x).group(0))
            x = x.lstrip('((!|\?)+)')
        else:
            p = re.match('\)|;|:|"|-|—|\*|,|\(', x).group(0)
            punct.append(p)
            x = x.lstrip(p)
    return x, punct

In [342]:
def rstrip(x):
    punct = []
    x1 = rev(x)
    while len(x1) > 0 and x1[0] in ['?', '!', '.', ')', '(', ';', ':', '"', '-', '—', '*' , ',']:
        if x1[0] == '.':
            if init(rev(x1)) != True and red(rev(x1)) != True:
                punct.append(re.match('\.+', x1).group(0)) #нашли все многоточия (в начале и в конце)
                x1 = x1.lstrip(re.match('\.+', x1).group(0)) #убрали многоточие
            else:
                break
        elif x1[0] == '?' or x1[0] == '!':
            punct.append(re.match('((!|\?)+)', x1).group(0))
            x1 = x1.lstrip(re.match('((!|\?)+)', x1).group(0))
            x = rev(x1)
        elif x1[0] == ')':
            punct.append(re.match('\)+-*:*', x1).group(0)) #проверка на веселые смайлики
            x1 = x1.lstrip(re.match('\)+-*:*', x1).group(0))
        elif x1[0] == '(':
            punct.append(re.match('\(+-*:*', x1).group(0)) #проверка на грустные смайлики
            x1 = x1.lstrip(re.match('\(+-*:*', x1).group(0))
        else:
            p = re.match(';|:|"|-|—|\*|,', x1).group(0)
            punct.append(p)
            x1 = x1.lstrip(p)
            x = rev(x1)
    num = re.search('([0-9]+\.{0,1}[а-яА-Яa-zA-Z]+)', x1)
    if num is not None and num.group(0) == x1:
        num1 = re.search('([0-9]+)(\.{0,1}[а-яА-Яa-zA-Z]+)', x1)
        x1 = num1.group(2)
        punct.append(num1.group(1))
    for i in range(len(punct)): #перевернуть скобки обратно
        punct[i] = rev(punct[i])
    return rev(x1), list(reversed(punct))

In [319]:
def red(x):
    reduces = ['д.', 'доб.', 'каб.', 'ул.', 'тел.', 'м.', 'пер.', 'руб.', 'e.g.', 'т.к.', 'гг.', 'г.', 'тыс.', 'тысч.', 'англ.', 'д.э.н.', 'к.ф.н.', 'д.ф.н.', 'ауд.', 'ноябр.', 'дек.', 'янв.', 'апр.', 'сент.', 'окт.', 'фев.', 'п.', 'к.', 'т.е.', 'л.', 'к.и.н.', 'см.', 'прил.', 'стр.', 'в.н.с.', 'мин.', 'М.', 'т.п.', 'P.S.']
    if x in reduces:
        return True
    else:
        return False

In [38]:
def init(x):
    flag = False
    s = re.search('([А-Я]\.[А-Я]\.)', x)
    if s is not None:
        if s.group(0) == x:
            flag = True
    return flag

In [311]:
def splitter(text):
    text = cleaner(text)
    words = text.split() #по умолчанию делим по пробелу, единые сущности, разделенные пробелом, будут рассмотрены потом
    words2 = []
    
    for x in range(len(words)):
            lstr = lstrip(words[x]) #стрип слева
            if len(lstr[1]) != 0:
                words2 += lstr[1] #знаки препинания перед словом
            rstr = rstrip(lstr[0]) #стрип справа
            if '(' in rstr[0]: #проверка на скобку внутри слова, часто перед ней нет пробела
                words2.append(rstr[0].split('(')[0])
                words2.append('(')
                words2.append(rstr[0].split('(')[1])
            else:
                words2.append(rstr[0]) #итоговое слово
            if len(rstr[1]) != 0:
                words2 += rstr[1] #знаки препинания после слова
                
    words3 = []
    for i in words2:
        if i != '':
            words3.append(i)
    return words3

In [346]:
cleaner(messages[32])

'Дорогие ребята, как вы знаете, в эту пятницу у вас последний шанс сдать переделанные работы. Но для этого необходимо забрать старые работы. Сегодня вы можете нати меня в университете до 16.40, завтра - весь день. Либо сами-молодцы заберете из 518 (первая полочка - Никитина). Если у вас есть какие-то вопросы, пишите! Всего доброго, Лена '

In [347]:
splitter(messages[32])

['Дорогие',
 'ребята',
 ',',
 'как',
 'вы',
 'знаете',
 ',',
 'в',
 'эту',
 'пятницу',
 'у',
 'вас',
 'последний',
 'шанс',
 'сдать',
 'переделанные',
 'работы',
 '.',
 'Но',
 'для',
 'этого',
 'необходимо',
 'забрать',
 'старые',
 'работы',
 '.',
 'Сегодня',
 'вы',
 'можете',
 'нати',
 'меня',
 'в',
 'университете',
 'до',
 '16.40',
 ',',
 'завтра',
 '-',
 'весь',
 'день',
 '.',
 'Либо',
 'сами-молодцы',
 'заберете',
 'из',
 '518',
 '(',
 'первая',
 'полочка',
 '-',
 'Никитина',
 ')',
 '.',
 'Если',
 'у',
 'вас',
 'есть',
 'какие-то',
 'вопросы',
 ',',
 'пишите',
 '!',
 'Всего',
 'доброго',
 ',',
 'Лена']